In [1]:
import pandas as pd
import statsmodels.api as sm
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import numpy as np

Importando a tabela de produção dos items da cesta básica (1974 - 2020)

In [2]:
producao = pd.read_excel('C:\dcc-fellowship-ciencia-de-dados\pam.xlsx')
producao.head()

,Ano,Item,Produçao
0,1974,Arroz,6764038
1,1975,Arroz,7781538
2,1976,Arroz,9757079
3,1977,Arroz,8993696
4,1978,Arroz,7296142


In [3]:
for item in producao.Item.unique():
    print(item)

Arroz 
Café (em grão)
Cana-de-açúcar
Feijão (em grão)
Mandioca
Milho (em grão)
Soja (em grão)
Tomate
Trigo (em grão)


Criando um dataframe para cada item da cesta

In [4]:
df_arroz=producao[producao['Item']=='Arroz '].reset_index(drop = True)
df_cafe=producao[producao['Item']=='Café (em grão)'].reset_index(drop = True)
df_cana=producao[producao['Item']=='Cana-de-açúcar'].reset_index(drop = True)
df_feijao=producao[producao['Item']=='Feijão (em grão)'].reset_index(drop = True)
df_mandioca=producao[producao['Item']=='Mandioca'].reset_index(drop = True)
df_milho=producao[producao['Item']=='Milho (em grão)'].reset_index(drop = True)
df_soja=producao[producao['Item']=='Soja (em grão)'].reset_index(drop = True)
df_tomate=producao[producao['Item']=='Tomate'].reset_index(drop = True)
df_trigo=producao[producao['Item']=='Trigo (em grão)'].reset_index(drop = True)

Cálculo da média móvel para cada item da cesta

* Arroz

In [5]:
## Média móvel
df_arroz['Média móvel'] = df_arroz['Produçao'].rolling(window =5).mean()

In [6]:
fig =px.line(df_arroz, x ='Ano', y = ['Produçao','Média móvel'], title = 'Produção de ARROZ - Brasil - (1974-2020)', labels= {'value': 'Produção (ton)'})
fig.update_layout(template = 'plotly_white')

In [7]:
df_arroz.to_excel('df_arroz_demanda.xlsx')

Regressão linear simples utilizando a coluna Média móvel como resposta

In [8]:
# Declarar as variáveis 
#Arroz
Y_arroz = df_arroz.loc[4:]['Média móvel'].values  
X_arroz = df_arroz.loc[4:][['Ano']].values
X_arroz = sm.add_constant(X_arroz)

In [9]:
#Arroz
modelo_arroz_mm = sm.OLS(Y_arroz,X_arroz)
resultados_arroz_mm = modelo_arroz_mm.fit()
#print(resultados_arroz_mm.summary())

In [10]:
resultados_arroz_mm.params[1]

95663.92869223907

In [11]:
print("Coeficientes: ", resultados_arroz_mm.params)
print("R2: ", resultados_arroz_mm.rsquared)

Coeficientes:  [-1.80831250e+08  9.56639287e+04]
R2:  0.8246475580115764


Filtrando os registros onde a coluna ANO é maior que 2018, já que estamos levando em consideração os padrões alimentares de consumo indicados pela POF (2018)

In [12]:
df_arroz = pd.read_excel("C:\dcc-fellowship-ciencia-de-dados\previsões consumo e producao\df_arroz_demanda.xlsx")
df_arroz = df_arroz[df_arroz['Ano']>2017]

Demanda populacional por ARROZ em toneladas (2018 -2030)

In [13]:
projecao_demanda_brasil = pd.read_excel("C:\dcc-fellowship-ciencia-de-dados\projecao_demanda_brasil.xlsx")
projecao_demanda_brasil.head()

,Unnamed: 0,Ano,Região,Populacao (mi),Items,Consumo médio per capita (kg),Demanda pop (ton)
0,0,2018,Brasil,208494900,Arroz polido,17.065000,3.557965e+06
1,1,2018,Brasil,208494900,Feijão,4.983333,1.039000e+06
2,2,2018,Brasil,208494900,Farinha de mandioca,2.030000,4.232446e+05
3,3,2018,Brasil,208494900,Farinha de trigo,2.311667,4.819707e+05
4,4,2018,Brasil,208494900,Fubá de milho,1.120000,2.335143e+05


In [14]:
df_demanda_arroz = projecao_demanda_brasil[projecao_demanda_brasil['Items'] == ' Arroz polido'][['Ano', 'Items', 'Região', 'Consumo médio per capita (kg)', 'Demanda pop (ton)', 'Populacao (mi)']].reset_index(drop = True)

Juntando os dataframe df_arroz e df_demanda_arroz

In [15]:
df_arroz = pd.merge(df_arroz, df_demanda_arroz, how = 'inner', on = 'Ano')

In [16]:
df_arroz['Razão Prod(mm)/Demanda'] = df_arroz['Produçao']/df_arroz['Demanda pop (ton)']
df_arroz['Razão Prod(mm)/População (kg/pct)'] = (df_arroz['Produçao']/df_arroz['Populacao (mi)'])*1000

In [17]:
df_arroz

,Ano,Item,Produçao,Items,Região,Consumo médio per capita (kg),Demanda pop (ton),Populacao (mi),Razão Prod(mm)/Demanda,Razão Prod(mm)/População (kg/pct)
0,2018,Arroz,1.187443e+07,Arroz polido,Brasil,17.065,3.557965e+06,208494900,3.337422,56.953115
1,2019,Arroz,1.151304e+07,Arroz polido,Brasil,17.065,3.586161e+06,210147125,3.210409,54.785624
2,2020,Arroz,1.127100e+07,Arroz polido,Brasil,17.065,3.613611e+06,211755692,3.119042,53.226448
3,2021,Arroz,1.136667e+07,Arroz polido,Brasil,17.065,3.640266e+06,213317639,3.122483,53.285169
4,2022,Arroz,1.167454e+07,Arroz polido,Brasil,17.065,3.666049e+06,214828540,3.184503,54.343544
5,2023,Arroz,1.163560e+07,Arroz polido,Brasil,17.065,3.690891e+06,216284269,3.152518,53.797722
6,2024,Arroz,1.158783e+07,Arroz polido,Brasil,17.065,3.714785e+06,217684462,3.119382,53.232253
7,2025,Arroz,1.160279e+07,Arroz polido,Brasil,17.065,3.737731e+06,219029093,3.104234,52.973752
8,2026,Arroz,1.166915e+07,Arroz polido,Brasil,17.065,3.759702e+06,220316530,3.103744,52.965389
9,2027,Arroz,1.172965e+07,Arroz polido,Brasil,17.065,3.780669e+06,221545234,3.102532,52.944708


In [18]:
fig =px.scatter(df_arroz, x ='Ano', y = ['Demanda pop (ton)', 'Produçao'], title = 'Projeção produção/demanda por ARROZ - Brasil - (2018-2030)', labels= {'value': 'Produção (ton)'}, trendline = 'ols')
fig.update_layout(template = 'plotly_white')


fig.update_layout(
    font_family="Helvetica",
    font_color="black",
    title_font_family="Helvetica",
    title_font_color="black",
    legend_title_font_color="black")

* Feijão

In [19]:
## Média móvel
df_feijao['Média móvel'] = df_feijao['Produçao'].rolling(window =5).mean()

In [20]:
fig =px.scatter(df_feijao, x ='Ano', y='Produçao', trendline = 'ols', title = 'Produção de FEIJÃO - Brasil - (1974-2020)', labels= {'value': 'Produção (ton)'})
fig.update_layout(template = 'plotly_white')

In [21]:
fig =px.scatter(df_feijao, x ='Ano', y='Média móvel', trendline = 'ols', title = 'Produção de FEIJÃO - Brasil - (1974-2020)', labels= {'value': 'Produção (ton)'})
fig.update_layout(template = 'plotly_white')

In [22]:
fig =px.line(df_feijao, x ='Ano', y=['Produçao','Média móvel'], title = 'Produção de FEIJÃO - Brasil - (1974-2020)', labels= {'value': 'Produção (ton)'})
fig.update_layout(template = 'plotly_white')

In [23]:
df_feijao.to_excel('df_feijao_demanda.xlsx')

Regressão linear simples utilizando a coluna Média móvel como variável resposta

In [24]:
# Declarar as variáveis 
#Arroz
Y_feijao= df_feijao.loc[4:]['Média móvel'].values  
X_feijao = df_feijao.loc[4:][['Ano']].values
X_feijao = sm.add_constant(X_feijao)

In [25]:
#Modelo
modelo_feijao_mm = sm.OLS(Y_feijao,X_feijao)
resultados_feijao_mm = modelo_feijao_mm.fit()
#print(resultados_feijao_mm.summary())

In [26]:
print("Coeficientes: ", resultados_feijao_mm.params)
print("R2: ", resultados_feijao_mm.rsquared)

Coeficientes:  [-5.05970273e+07  2.66786866e+04]
R2:  0.7875152128257473


In [27]:
resultados_feijao_mm.params[1]

26678.686590154357

Filtrando os registros onde a coluna ANO é maior que 2018, já que estamos levando em consideração os padrões alimentares de consumo indicados pela POF (2018)

In [28]:
df_feijao = pd.read_excel("C:\dcc-fellowship-ciencia-de-dados\previsões consumo e producao\df_feijao_demanda.xlsx")
df_feijao = df_feijao[df_feijao['Ano']>2017].reset_index(drop = True)
df_feijao

,Ano,Item,Produção ton
0,2018,Feijão,2.993445e+06
1,2019,Feijão,2.916142e+06
2,2020,Feijão,2.905415e+06
3,2021,Feijão,2.932093e+06
4,2022,Feijão,2.973835e+06
5,2023,Feijão,2.970864e+06
6,2024,Feijão,2.966348e+06
7,2025,Feijão,2.976389e+06
8,2026,Feijão,2.990584e+06
9,2027,Feijão,3.002282e+06


Demanda populacional por FEIJÂO em toneladas (2018 -2030)

In [29]:
df_demanda_feijao = projecao_demanda_brasil[projecao_demanda_brasil['Items'] == 'Feijão'][['Ano', 'Items', 'Região', 'Demanda pop (ton)', 'Populacao (mi)', 'Consumo médio per capita (kg)']].reset_index(drop = True)
df_demanda_feijao

,Ano,Items,Região,Demanda pop (ton),Populacao (mi),Consumo médio per capita (kg)
0,2018,Feijão,Brasil,1.039000e+06,208494900,4.983333
1,2019,Feijão,Brasil,1.047233e+06,210147125,4.983333
2,2020,Feijão,Brasil,1.055249e+06,211755692,4.983333
3,2021,Feijão,Brasil,1.063033e+06,213317639,4.983333
4,2022,Feijão,Brasil,1.070562e+06,214828540,4.983333
5,2023,Feijão,Brasil,1.077817e+06,216284269,4.983333
6,2024,Feijão,Brasil,1.084794e+06,217684462,4.983333
7,2025,Feijão,Brasil,1.091495e+06,219029093,4.983333
8,2026,Feijão,Brasil,1.097911e+06,220316530,4.983333
9,2027,Feijão,Brasil,1.104034e+06,221545234,4.983333


Juntando os dataframe df_feijao e df_demanda_feijao

In [30]:
df_feijao = pd.merge(df_feijao, df_demanda_feijao, how = 'inner', on = 'Ano')
df_feijao

,Ano,Item,Produção ton,Items,Região,Demanda pop (ton),Populacao (mi),Consumo médio per capita (kg)
0,2018,Feijão,2.993445e+06,Feijão,Brasil,1.039000e+06,208494900,4.983333
1,2019,Feijão,2.916142e+06,Feijão,Brasil,1.047233e+06,210147125,4.983333
2,2020,Feijão,2.905415e+06,Feijão,Brasil,1.055249e+06,211755692,4.983333
3,2021,Feijão,2.932093e+06,Feijão,Brasil,1.063033e+06,213317639,4.983333
4,2022,Feijão,2.973835e+06,Feijão,Brasil,1.070562e+06,214828540,4.983333
5,2023,Feijão,2.970864e+06,Feijão,Brasil,1.077817e+06,216284269,4.983333
6,2024,Feijão,2.966348e+06,Feijão,Brasil,1.084794e+06,217684462,4.983333
7,2025,Feijão,2.976389e+06,Feijão,Brasil,1.091495e+06,219029093,4.983333
8,2026,Feijão,2.990584e+06,Feijão,Brasil,1.097911e+06,220316530,4.983333
9,2027,Feijão,3.002282e+06,Feijão,Brasil,1.104034e+06,221545234,4.983333


In [31]:
df_feijao['Razão Prod(mm)/Demanda'] = df_feijao['Produção ton']/df_feijao['Demanda pop (ton)']
df_feijao['Razão Prod(mm)/População (kg/pct)'] = (df_feijao['Produção ton']/df_feijao['Populacao (mi)'])*1000

In [32]:
df_feijao

,Ano,Item,Produção ton,Items,Região,Demanda pop (ton),Populacao (mi),Consumo médio per capita (kg),Razão Prod(mm)/Demanda,Razão Prod(mm)/População (kg/pct)
0,2018,Feijão,2.993445e+06,Feijão,Brasil,1.039000e+06,208494900,4.983333,2.881084,14.357400
1,2019,Feijão,2.916142e+06,Feijão,Brasil,1.047233e+06,210147125,4.983333,2.784616,13.876670
2,2020,Feijão,2.905415e+06,Feijão,Brasil,1.055249e+06,211755692,4.983333,2.753298,13.720600
3,2021,Feijão,2.932093e+06,Feijão,Brasil,1.063033e+06,213317639,4.983333,2.758234,13.745198
4,2022,Feijão,2.973835e+06,Feijão,Brasil,1.070562e+06,214828540,4.983333,2.777826,13.842832
5,2023,Feijão,2.970864e+06,Feijão,Brasil,1.077817e+06,216284269,4.983333,2.756373,13.735923
6,2024,Feijão,2.966348e+06,Feijão,Brasil,1.084794e+06,217684462,4.983333,2.734480,13.626825
7,2025,Feijão,2.976389e+06,Feijão,Brasil,1.091495e+06,219029093,4.983333,2.726892,13.589013
8,2026,Feijão,2.990584e+06,Feijão,Brasil,1.097911e+06,220316530,4.983333,2.723886,13.574034
9,2027,Feijão,3.002282e+06,Feijão,Brasil,1.104034e+06,221545234,4.983333,2.719375,13.551554


In [33]:
fig =px.scatter(df_feijao, x ='Ano', y = ['Demanda pop (ton)', 'Produção ton'], title = 'Projeção da razão produção/demanda por FEIJÃO - Brasil - (2018-2030)', labels= {'value': 'Produção (ton)'}, trendline = 'ols')
fig.update_layout(template = 'plotly_white')


fig.update_layout(
    font_family="Helvetica",
    font_color="black",
    title_font_family="Helvetica",
    title_font_color="black",
    legend_title_font_color="black")

* Cana

In [34]:
## Média móvel
df_cana['Média móvel'] = df_cana['Produçao'].rolling(window =5).mean()

In [35]:
fig =px.line(df_cana, x ='Ano', y=['Produçao', 'Média móvel'], title = 'Produção de CANA-DE-AÇUCAR - Brasil - (1974-2020)', labels= {'value': 'Produção (ton)'})
fig.update_layout(template = 'plotly_white')

In [36]:
df_cana.to_excel('df_cana_demanda.xlsx')

Regressão linear simples utilizando a coluna Média Móvel como demanda

In [37]:
# Declarar as variáveis 
#Cana
Y_cana= df_cana.loc[4:]['Média móvel'].values  
X_cana = df_cana.loc[4:][['Ano']].values
X_cana = sm.add_constant(X_cana)

In [38]:
#Modelo
modelo_cana = sm.OLS(Y_cana,X_cana)
resultados_cana = modelo_cana.fit()
#print(resultados_cana_mm.summary())

In [39]:
print("Coeficientes: ", resultados_cana.params)
print("R2: ", resultados_cana.rsquared)

Coeficientes:  [-3.26436167e+10  1.65275074e+07]
R2:  0.9119548158915369


In [40]:
resultados_cana.params[1]

16527507.355512176

Filtrando os registros onde a coluna ANO é maior que 2018, já que estamos levando em consideração os padrões alimentares de consumo indicados pela POF (2018)

In [41]:
df_cana = pd.read_excel("C:\dcc-fellowship-ciencia-de-dados\previsões consumo e producao\df_cana_demanda.xlsx")
df_cana = df_cana[df_cana['Ano']>2017].reset_index(drop = True)
df_cana

,Ano,Item,Prod ton
0,2018,Cana-de-açúcar,7.522392e+08
1,2019,Cana-de-açúcar,7.557116e+08
2,2020,Cana-de-açúcar,7.570769e+08
3,2021,Cana-de-açúcar,7.736044e+08
4,2022,Cana-de-açúcar,7.755231e+08
5,2023,Cana-de-açúcar,7.793585e+08
6,2024,Cana-de-açúcar,7.847824e+08
7,2025,Cana-de-açúcar,7.905966e+08
8,2026,Cana-de-açúcar,7.973005e+08
9,2027,Cana-de-açúcar,8.020397e+08


Correção em relação a proporção da produção de CANA destinada para a produção e consumo doméstico de AÇÚCAR CRISTAL

In [42]:
df_cana['Prod ton'] = df_cana['Prod ton']*0.062

In [43]:
df_cana

,Ano,Item,Prod ton
0,2018,Cana-de-açúcar,4.663883e+07
1,2019,Cana-de-açúcar,4.685412e+07
2,2020,Cana-de-açúcar,4.693877e+07
3,2021,Cana-de-açúcar,4.796347e+07
4,2022,Cana-de-açúcar,4.808243e+07
5,2023,Cana-de-açúcar,4.832023e+07
6,2024,Cana-de-açúcar,4.865651e+07
7,2025,Cana-de-açúcar,4.901699e+07
8,2026,Cana-de-açúcar,4.943263e+07
9,2027,Cana-de-açúcar,4.972646e+07


Demanda populacional por CANA em toneladas (2018 -2030)

In [44]:
df_demanda_acucar = projecao_demanda_brasil[projecao_demanda_brasil['Items'] == 'Açúcar cristal'][['Ano', 'Items', 'Região', 'Demanda pop (ton)', 'Populacao (mi)', 'Consumo médio per capita (kg)']].reset_index(drop = True)
df_demanda_acucar

,Ano,Items,Região,Demanda pop (ton),Populacao (mi),Consumo médio per capita (kg)
0,2018,Açúcar cristal,Brasil,1.141510e+06,208494900,5.475
1,2019,Açúcar cristal,Brasil,1.150556e+06,210147125,5.475
2,2020,Açúcar cristal,Brasil,1.159362e+06,211755692,5.475
3,2021,Açúcar cristal,Brasil,1.167914e+06,213317639,5.475
4,2022,Açúcar cristal,Brasil,1.176186e+06,214828540,5.475
5,2023,Açúcar cristal,Brasil,1.184156e+06,216284269,5.475
6,2024,Açúcar cristal,Brasil,1.191822e+06,217684462,5.475
7,2025,Açúcar cristal,Brasil,1.199184e+06,219029093,5.475
8,2026,Açúcar cristal,Brasil,1.206233e+06,220316530,5.475
9,2027,Açúcar cristal,Brasil,1.212960e+06,221545234,5.475


Juntando os dataframe df_cana e df_demanda_cana

In [45]:
df_cana = pd.merge(df_cana, df_demanda_acucar, how = 'inner', on = 'Ano')
df_cana

,Ano,Item,Prod ton,Items,Região,Demanda pop (ton),Populacao (mi),Consumo médio per capita (kg)
0,2018,Cana-de-açúcar,4.663883e+07,Açúcar cristal,Brasil,1.141510e+06,208494900,5.475
1,2019,Cana-de-açúcar,4.685412e+07,Açúcar cristal,Brasil,1.150556e+06,210147125,5.475
2,2020,Cana-de-açúcar,4.693877e+07,Açúcar cristal,Brasil,1.159362e+06,211755692,5.475
3,2021,Cana-de-açúcar,4.796347e+07,Açúcar cristal,Brasil,1.167914e+06,213317639,5.475
4,2022,Cana-de-açúcar,4.808243e+07,Açúcar cristal,Brasil,1.176186e+06,214828540,5.475
5,2023,Cana-de-açúcar,4.832023e+07,Açúcar cristal,Brasil,1.184156e+06,216284269,5.475
6,2024,Cana-de-açúcar,4.865651e+07,Açúcar cristal,Brasil,1.191822e+06,217684462,5.475
7,2025,Cana-de-açúcar,4.901699e+07,Açúcar cristal,Brasil,1.199184e+06,219029093,5.475
8,2026,Cana-de-açúcar,4.943263e+07,Açúcar cristal,Brasil,1.206233e+06,220316530,5.475
9,2027,Cana-de-açúcar,4.972646e+07,Açúcar cristal,Brasil,1.212960e+06,221545234,5.475


In [46]:
df_cana['Razão Prod(mm)/Demanda'] = df_cana['Prod ton']/df_cana['Demanda pop (ton)']
df_cana['Razão Prod(mm)/População (kg/pct)'] = (df_cana['Prod ton']/df_cana['Populacao (mi)'])*1000

In [47]:
df_cana

,Ano,Item,Prod ton,Items,Região,Demanda pop (ton),Populacao (mi),Consumo médio per capita (kg),Razão Prod(mm)/Demanda,Razão Prod(mm)/População (kg/pct)
0,2018,Cana-de-açúcar,4.663883e+07,Açúcar cristal,Brasil,1.141510e+06,208494900,5.475,40.857151,223.692901
1,2019,Cana-de-açúcar,4.685412e+07,Açúcar cristal,Brasil,1.150556e+06,210147125,5.475,40.723040,222.958642
2,2020,Cana-de-açúcar,4.693877e+07,Açúcar cristal,Brasil,1.159362e+06,211755692,5.475,40.486708,221.664725
3,2021,Cana-de-açúcar,4.796347e+07,Açúcar cristal,Brasil,1.167914e+06,213317639,5.475,41.067638,224.845319
4,2022,Cana-de-açúcar,4.808243e+07,Açúcar cristal,Brasil,1.176186e+06,214828540,5.475,40.879947,223.817711
5,2023,Cana-de-açúcar,4.832023e+07,Açúcar cristal,Brasil,1.184156e+06,216284269,5.475,40.805615,223.410743
6,2024,Cana-de-açúcar,4.865651e+07,Açúcar cristal,Brasil,1.191822e+06,217684462,5.475,40.825301,223.518522
7,2025,Cana-de-açúcar,4.901699e+07,Açúcar cristal,Brasil,1.199184e+06,219029093,5.475,40.875275,223.792131
8,2026,Cana-de-açúcar,4.943263e+07,Açúcar cristal,Brasil,1.206233e+06,220316530,5.475,40.980997,224.370961
9,2027,Cana-de-açúcar,4.972646e+07,Açúcar cristal,Brasil,1.212960e+06,221545234,5.475,40.995958,224.452869


In [130]:
fig =px.scatter(df_cana, x ='Ano', y = ['Prod ton', 'Demanda pop (ton)'], title = 'Projeção produção/demanda por CANA-DE-AÇUCAR/AÇUCAR CRISTAL - Brasil - (2018-2030)', labels= {'value': 'Produção (ton)'}, trendline = 'ols')
fig.update_layout(template = 'plotly_white')


fig.update_layout(
    font_family="Helvetica",
    font_color="black",
    title_font_family="Helvetica",
    title_font_color="black",
    legend_title_font_color="black")

* Café

In [49]:
## Média móvel
df_cafe['Média móvel'] = df_cafe['Produçao'].rolling(window =5).mean()

In [50]:
fig =px.line(df_cafe, x ='Ano', y=['Produçao', 'Média móvel'], title = 'Produção de CAFÉ - Brasil - (1974-2020)', labels= {'value': 'Produção (ton)'})
fig.update_layout(template = 'plotly_white')

In [51]:
df_cafe.to_excel('df_cafe_demanda.xlsx')

Regressão linear simples utilizando a coluna média movel como resposta

In [52]:
# Declarar as variáveis 
#Cana
Y_cafe= df_cafe.loc[4:]['Média móvel'].values  
X_cafe = df_cafe.loc[4:][['Ano']].values
X_cafe = sm.add_constant(X_cana)

In [53]:
#Modelo
modelo_cafe = sm.OLS(Y_cafe,X_cafe)
resultados_cafe = modelo_cafe.fit()
#print(resultados_cana.summary())

In [54]:
print("Coeficientes: ", resultados_cafe.params)
print("R2: ", resultados_cafe.rsquared)

Coeficientes:  [-4.84865285e+06  3.81830936e+03]
R2:  0.02068642534011156


In [55]:
resultados_cafe.params[1]

3818.309362730035

Filtrando os registros onde a coluna ANO é maior que 2018, já que estamos levando em consideração os padrões alimentares de consumo indicados pela POF (2018)

In [56]:
df_cafe = pd.read_excel("C:\dcc-fellowship-ciencia-de-dados\previsões consumo e producao\df_cafe_demanda.xlsx")
df_cafe = df_cafe[df_cafe['Ano']>2017].reset_index(drop = True)
df_cafe

,Ano,Item,Média móvel
0,2018,Café (em grão),2.942655e+06
1,2019,Café (em grão),2.984190e+06
2,2020,Café (em grão),3.194736e+06
3,2021,Café (em grão),3.198554e+06
4,2022,Café (em grão),3.032849e+06
5,2023,Café (em grão),3.074415e+06
6,2024,Café (em grão),3.100767e+06
7,2025,Café (em grão),3.124082e+06
8,2026,Café (em grão),3.109951e+06
9,2027,Café (em grão),3.092231e+06


Demanda populacional por CAFÉ SOLÚVEL em toneladas (2018 -2030)

In [57]:
df_demanda_cafe = projecao_demanda_brasil[projecao_demanda_brasil['Items'] == 'Café moído'][['Ano', 'Items', 'Região', 'Demanda pop (ton)', 'Populacao (mi)', 'Consumo médio per capita (kg)']].reset_index(drop = True)
df_demanda_cafe

,Ano,Items,Região,Demanda pop (ton),Populacao (mi),Consumo médio per capita (kg)
0,2018,Café moído,Brasil,510465.013500,208494900,2.448333
1,2019,Café moído,Brasil,514510.211042,210147125,2.448333
2,2020,Café moído,Brasil,518448.519247,211755692,2.448333
3,2021,Café moído,Brasil,522272.686152,213317639,2.448333
4,2022,Café moído,Brasil,525971.875433,214828540,2.448333
5,2023,Café moído,Brasil,529535.985268,216284269,2.448333
6,2024,Café moído,Brasil,532964.124463,217684462,2.448333
7,2025,Café moído,Brasil,536256.229362,219029093,2.448333
8,2026,Café moído,Brasil,539408.304283,220316530,2.448333
9,2027,Café moído,Brasil,542416.581243,221545234,2.448333


Juntando os dataframe df_cafe e df_demanda_cafe

In [58]:
df_cafe = pd.merge(df_cafe, df_demanda_cafe, how = 'inner', on = 'Ano')
df_cafe

,Ano,Item,Média móvel,Items,Região,Demanda pop (ton),Populacao (mi),Consumo médio per capita (kg)
0,2018,Café (em grão),2.942655e+06,Café moído,Brasil,510465.013500,208494900,2.448333
1,2019,Café (em grão),2.984190e+06,Café moído,Brasil,514510.211042,210147125,2.448333
2,2020,Café (em grão),3.194736e+06,Café moído,Brasil,518448.519247,211755692,2.448333
3,2021,Café (em grão),3.198554e+06,Café moído,Brasil,522272.686152,213317639,2.448333
4,2022,Café (em grão),3.032849e+06,Café moído,Brasil,525971.875433,214828540,2.448333
5,2023,Café (em grão),3.074415e+06,Café moído,Brasil,529535.985268,216284269,2.448333
6,2024,Café (em grão),3.100767e+06,Café moído,Brasil,532964.124463,217684462,2.448333
7,2025,Café (em grão),3.124082e+06,Café moído,Brasil,536256.229362,219029093,2.448333
8,2026,Café (em grão),3.109951e+06,Café moído,Brasil,539408.304283,220316530,2.448333
9,2027,Café (em grão),3.092231e+06,Café moído,Brasil,542416.581243,221545234,2.448333


In [59]:
df_cafe['Razão Prod(mm)/Demanda'] = df_cafe['Média móvel']/df_cafe['Demanda pop (ton)']
df_cafe['Razão Prod(mm)/População (kg/pct)'] = (df_cafe['Média móvel']/df_cafe['Populacao (mi)'])*1000
df_cafe

,Ano,Item,Média móvel,Items,Região,Demanda pop (ton),Populacao (mi),Consumo médio per capita (kg),Razão Prod(mm)/Demanda,Razão Prod(mm)/População (kg/pct)
0,2018,Café (em grão),2.942655e+06,Café moído,Brasil,510465.013500,208494900,2.448333,5.764656,14.113800
1,2019,Café (em grão),2.984190e+06,Café moído,Brasil,514510.211042,210147125,2.448333,5.800061,14.200482
2,2020,Café (em grão),3.194736e+06,Café moído,Brasil,518448.519247,211755692,2.448333,6.162108,15.086895
3,2021,Café (em grão),3.198554e+06,Café moído,Brasil,522272.686152,213317639,2.448333,6.124298,14.994324
4,2022,Café (em grão),3.032849e+06,Café moído,Brasil,525971.875433,214828540,2.448333,5.766180,14.117531
5,2023,Café (em grão),3.074415e+06,Café moído,Brasil,529535.985268,216284269,2.448333,5.805866,14.214694
6,2024,Café (em grão),3.100767e+06,Café moído,Brasil,532964.124463,217684462,2.448333,5.817965,14.244318
7,2025,Café (em grão),3.124082e+06,Café moído,Brasil,536256.229362,219029093,2.448333,5.825726,14.263319
8,2026,Café (em grão),3.109951e+06,Café moído,Brasil,539408.304283,220316530,2.448333,5.765486,14.115832
9,2027,Café (em grão),3.092231e+06,Café moído,Brasil,542416.581243,221545234,2.448333,5.700841,13.957559


In [131]:
fig =px.scatter(df_cafe, x ='Ano', y = ['Média móvel', 'Demanda pop (ton)'], title = 'Projeção da produção/demanda por CAFÉ/CAFÉ MOÍDO - Brasil - (2018-2030)', labels= {'value': 'Produção (ton)'}, trendline = 'ols')
fig.update_layout(template = 'plotly_white')


fig.update_layout(
    font_family="Helvetica",
    font_color="black",
    title_font_family="Helvetica",
    title_font_color="black",
    legend_title_font_color="black")

* Mandioca

In [61]:
## Média móvel
df_mandioca['Média móvel'] = df_mandioca['Produçao'].rolling(window =5).mean()

In [62]:
fig =px.line(df_mandioca, x ='Ano', y=['Produçao', 'Média móvel'], title = 'Produção de MANDIOCA - Brasil - (1974-2020)', labels= {'value': 'Produção (ton)'})
fig.update_layout(template = 'plotly_white')

In [63]:
df_mandioca.to_excel('df_mandioca_demada.xlsx')

Regressão linear simples utilizando a coluna média movel como resposta

In [64]:
# Declarar as variáveis 
#Cana
Y_mandioca= df_mandioca.loc[4:]['Média móvel'].values  
X_mandioca = df_mandioca.loc[4:][['Ano']].values
X_mandioca = sm.add_constant(X_mandioca)

In [65]:
#Mandioca
modelo_mandioca = sm.OLS(Y_mandioca,X_mandioca)
resultados_mandioca = modelo_mandioca.fit()
#print(resultados_mandioca.summary())

In [66]:
print("Coeficientes: ", resultados_mandioca.params)
print("R2: ", resultados_mandioca.rsquared)

Coeficientes:  [ 1.12570415e+08 -4.46970284e+04]
R2:  0.10359849310247826


In [67]:
resultados_mandioca.params[1]

-44697.02842041658

In [68]:
df_mandioca.columns

Index(['Ano', 'Item', 'Produçao', 'Média móvel'], dtype='object')

Filtrando os registros onde a coluna ANO é maior que 2018, já que estamos levando em consideração os padrões alimentares de consumo indicados pela POF (2018)

In [69]:
df_mandioca = pd.read_excel("C:\dcc-fellowship-ciencia-de-dados\previsões consumo e producao\df_mandioca_demanda.xlsx")
df_mandioca = df_mandioca[df_mandioca['Ano']>2017].reset_index(drop = True)
df_mandioca

,Ano,Item,Média móvel
0,2018,Mandioca,2.074567e+07
1,2019,Mandioca,1.961360e+07
2,2020,Mandioca,1.864269e+07
3,2021,Mandioca,1.859799e+07
4,2022,Mandioca,1.976871e+07
5,2023,Mandioca,1.942903e+07
6,2024,Mandioca,1.916571e+07
7,2025,Mandioca,1.907613e+07
8,2026,Mandioca,1.916282e+07
9,2027,Mandioca,1.927578e+07


Correção em relação a proporção da produção de MANDIOCA destinada para a produção e consumo doméstico de FARINHA DE MANDIOCA

In [70]:
df_mandioca['Média móvel'] = df_mandioca['Média móvel']*0.75

Demanda populacional por FARINHA DE MANDIOCA em toneladas (2018 -2030)

In [71]:
df_demanda_farinha_mandioca = projecao_demanda_brasil[projecao_demanda_brasil['Items'] == 'Farinha de mandioca'][['Ano', 'Items', 'Região', 'Demanda pop (ton)', 'Populacao (mi)', 'Consumo médio per capita (kg)']].reset_index(drop = True)
df_demanda_farinha_mandioca

,Ano,Items,Região,Demanda pop (ton),Populacao (mi),Consumo médio per capita (kg)
0,2018,Farinha de mandioca,Brasil,423244.64700,208494900,2.03
1,2019,Farinha de mandioca,Brasil,426598.66375,210147125,2.03
2,2020,Farinha de mandioca,Brasil,429864.05476,211755692,2.03
3,2021,Farinha de mandioca,Brasil,433034.80717,213317639,2.03
4,2022,Farinha de mandioca,Brasil,436101.93620,214828540,2.03
5,2023,Farinha de mandioca,Brasil,439057.06607,216284269,2.03
6,2024,Farinha de mandioca,Brasil,441899.45786,217684462,2.03
7,2025,Farinha de mandioca,Brasil,444629.05879,219029093,2.03
8,2026,Farinha de mandioca,Brasil,447242.55590,220316530,2.03
9,2027,Farinha de mandioca,Brasil,449736.82502,221545234,2.03


Juntando os dataframe df_mandioca e df_demanda_farinha_mandioca

In [72]:
df_mandioca = pd.merge(df_mandioca, df_demanda_farinha_mandioca, how = 'inner', on = 'Ano')
df_mandioca

,Ano,Item,Média móvel,Items,Região,Demanda pop (ton),Populacao (mi),Consumo médio per capita (kg)
0,2018,Mandioca,1.555925e+07,Farinha de mandioca,Brasil,423244.64700,208494900,2.03
1,2019,Mandioca,1.471020e+07,Farinha de mandioca,Brasil,426598.66375,210147125,2.03
2,2020,Mandioca,1.398201e+07,Farinha de mandioca,Brasil,429864.05476,211755692,2.03
3,2021,Mandioca,1.394849e+07,Farinha de mandioca,Brasil,433034.80717,213317639,2.03
4,2022,Mandioca,1.482653e+07,Farinha de mandioca,Brasil,436101.93620,214828540,2.03
5,2023,Mandioca,1.457178e+07,Farinha de mandioca,Brasil,439057.06607,216284269,2.03
6,2024,Mandioca,1.437428e+07,Farinha de mandioca,Brasil,441899.45786,217684462,2.03
7,2025,Mandioca,1.430710e+07,Farinha de mandioca,Brasil,444629.05879,219029093,2.03
8,2026,Mandioca,1.437211e+07,Farinha de mandioca,Brasil,447242.55590,220316530,2.03
9,2027,Mandioca,1.445684e+07,Farinha de mandioca,Brasil,449736.82502,221545234,2.03


In [73]:
df_mandioca['Razão Prod(mm)/Demanda'] = df_mandioca['Média móvel']/df_mandioca['Demanda pop (ton)']
df_mandioca['Razão Prod(mm)/População (kg/pct)'] = (df_mandioca['Média móvel']/df_mandioca['Populacao (mi)'])*1000
df_mandioca

,Ano,Item,Média móvel,Items,Região,Demanda pop (ton),Populacao (mi),Consumo médio per capita (kg),Razão Prod(mm)/Demanda,Razão Prod(mm)/População (kg/pct)
0,2018,Mandioca,1.555925e+07,Farinha de mandioca,Brasil,423244.64700,208494900,2.03,36.761838,74.626530
1,2019,Mandioca,1.471020e+07,Farinha de mandioca,Brasil,426598.66375,210147125,2.03,34.482531,69.999539
2,2020,Mandioca,1.398201e+07,Farinha de mandioca,Brasil,429864.05476,211755692,2.03,32.526595,66.028989
3,2021,Mandioca,1.394849e+07,Farinha de mandioca,Brasil,433034.80717,213317639,2.03,32.211017,65.388364
4,2022,Mandioca,1.482653e+07,Farinha de mandioca,Brasil,436101.93620,214828540,2.03,33.997856,69.015648
5,2023,Mandioca,1.457178e+07,Farinha de mandioca,Brasil,439057.06607,216284269,2.03,33.188795,67.373254
6,2024,Mandioca,1.437428e+07,Farinha de mandioca,Brasil,441899.45786,217684462,2.03,32.528395,66.032641
7,2025,Mandioca,1.430710e+07,Farinha de mandioca,Brasil,444629.05879,219029093,2.03,32.177599,65.320526
8,2026,Mandioca,1.437211e+07,Farinha de mandioca,Brasil,447242.55590,220316530,2.03,32.134938,65.233924
9,2027,Mandioca,1.445684e+07,Farinha de mandioca,Brasil,449736.82502,221545234,2.03,32.145101,65.254556


In [74]:
df_mandioca.columns

Index(['Ano', 'Item', 'Média móvel', 'Items', 'Região', 'Demanda pop (ton)',
       'Populacao (mi)', 'Consumo médio per capita (kg)',
       'Razão Prod(mm)/Demanda', 'Razão Prod(mm)/População (kg/pct)'],
      dtype='object')

In [132]:
fig =px.scatter(df_mandioca, x ='Ano', y = ['Média móvel', 'Demanda pop (ton)'], title = 'Projeção da produção/demanda por MANDIOCA/FARINHA DE MANDIOCA - Brasil - (2018-2030)', labels= {'value': 'Produção (ton)'}, trendline = 'ols')
fig.update_layout(template = 'plotly_white')

fig.update_layout(
    font_family="Helvetica",
    font_color="black",
    title_font_family="Helvetica",
    title_font_color="black",
    legend_title_font_color="black")

* Milho

In [76]:
## Média móvel
df_milho['Média móvel'] = df_milho['Produçao'].rolling(window =5).mean()

In [77]:
fig =px.line(df_milho, x ='Ano', y=['Produçao', 'Média móvel'], title = 'Produção de MILHO - Brasil - (1974-2020)', labels= {'value': 'Produção (ton)'})
fig.update_layout(template = 'plotly_white')

In [78]:
df_milho.to_excel('df_milho_demanda.xlsx')

In [79]:
# Declarar as variáveis 
#Milho
Y_milho= df_milho.loc[4:]['Média móvel'].values  
X_milho = df_milho.loc[4:][['Ano']].values
X_milho= sm.add_constant(X_milho)

In [80]:
#Milho
modelo_milho = sm.OLS(Y_milho,X_milho)
resultados_milho = modelo_milho.fit()
#print(resultados_milho.summary())

In [81]:
print("Coeficientes: ", resultados_milho.params)
print("R2: ", resultados_milho.rsquared)

Coeficientes:  [-3.14034490e+09  1.59083931e+06]
R2:  0.8741085894428411


In [82]:
resultados_milho.params[1]

1590839.3149199905

Filtrando os registros onde a coluna ANO é maior que 2018, já que estamos levando em consideração os padrões alimentares de consumo indicados pela POF (2018)

In [83]:
df_milho = pd.read_excel("C:\dcc-fellowship-ciencia-de-dados\previsões consumo e producao\df_milho_demanda.xlsx")
df_milho = df_milho[df_milho['Ano']>2017].reset_index(drop = True)
df_milho

,Ano,Item,Média móvel
0,2018,Milho (em grão),8.192604e+07
1,2019,Milho (em grão),8.617500e+07
2,2020,Milho (em grão),8.991111e+07
3,2021,Milho (em grão),9.150195e+07
4,2022,Milho (em grão),8.779513e+07
5,2023,Milho (em grão),8.905268e+07
6,2024,Milho (em grão),9.047801e+07
7,2025,Milho (em grão),9.133861e+07
8,2026,Milho (em grão),9.162412e+07
9,2027,Milho (em grão),9.164855e+07


Correção em relação a proporção da produção de MILHO destinada para a produção e consumo doméstico de FUBÁ DE MILHO

In [84]:
df_milho['Média móvel'] = df_milho['Média móvel']*0.025

In [85]:
df_milho

,Ano,Item,Média móvel
0,2018,Milho (em grão),2.048151e+06
1,2019,Milho (em grão),2.154375e+06
2,2020,Milho (em grão),2.247778e+06
3,2021,Milho (em grão),2.287549e+06
4,2022,Milho (em grão),2.194878e+06
5,2023,Milho (em grão),2.226317e+06
6,2024,Milho (em grão),2.261950e+06
7,2025,Milho (em grão),2.283465e+06
8,2026,Milho (em grão),2.290603e+06
9,2027,Milho (em grão),2.291214e+06


Demanda populacional por FUBÁ DE MILHO em toneladas (2018 -2030)

In [86]:
df_demanda_fuba_milho = projecao_demanda_brasil[projecao_demanda_brasil['Items'] == 'Fubá de milho'][['Ano', 'Items', 'Região', 'Demanda pop (ton)', 'Populacao (mi)', 'Consumo médio per capita (kg)']].reset_index(drop = True)
df_demanda_fuba_milho

,Ano,Items,Região,Demanda pop (ton),Populacao (mi),Consumo médio per capita (kg)
0,2018,Fubá de milho,Brasil,233514.28800,208494900,1.12
1,2019,Fubá de milho,Brasil,235364.78000,210147125,1.12
2,2020,Fubá de milho,Brasil,237166.37504,211755692,1.12
3,2021,Fubá de milho,Brasil,238915.75568,213317639,1.12
4,2022,Fubá de milho,Brasil,240607.96480,214828540,1.12
5,2023,Fubá de milho,Brasil,242238.38128,216284269,1.12
6,2024,Fubá de milho,Brasil,243806.59744,217684462,1.12
7,2025,Fubá de milho,Brasil,245312.58416,219029093,1.12
8,2026,Fubá de milho,Brasil,246754.51360,220316530,1.12
9,2027,Fubá de milho,Brasil,248130.66208,221545234,1.12


Juntando os dataframe df_milho e df_demanda_fuba_milho

In [87]:
df_milho = pd.merge(df_milho, df_demanda_fuba_milho, how = 'inner', on = 'Ano')
df_milho

,Ano,Item,Média móvel,Items,Região,Demanda pop (ton),Populacao (mi),Consumo médio per capita (kg)
0,2018,Milho (em grão),2.048151e+06,Fubá de milho,Brasil,233514.28800,208494900,1.12
1,2019,Milho (em grão),2.154375e+06,Fubá de milho,Brasil,235364.78000,210147125,1.12
2,2020,Milho (em grão),2.247778e+06,Fubá de milho,Brasil,237166.37504,211755692,1.12
3,2021,Milho (em grão),2.287549e+06,Fubá de milho,Brasil,238915.75568,213317639,1.12
4,2022,Milho (em grão),2.194878e+06,Fubá de milho,Brasil,240607.96480,214828540,1.12
5,2023,Milho (em grão),2.226317e+06,Fubá de milho,Brasil,242238.38128,216284269,1.12
6,2024,Milho (em grão),2.261950e+06,Fubá de milho,Brasil,243806.59744,217684462,1.12
7,2025,Milho (em grão),2.283465e+06,Fubá de milho,Brasil,245312.58416,219029093,1.12
8,2026,Milho (em grão),2.290603e+06,Fubá de milho,Brasil,246754.51360,220316530,1.12
9,2027,Milho (em grão),2.291214e+06,Fubá de milho,Brasil,248130.66208,221545234,1.12


In [88]:
df_milho['Razão Prod(mm)/Demanda'] = df_milho['Média móvel']/df_milho['Demanda pop (ton)']
df_milho['Razão Prod(mm)/População (kg/pct)'] = (df_milho['Média móvel']/df_milho['Populacao (mi)'])*1000
df_milho

,Ano,Item,Média móvel,Items,Região,Demanda pop (ton),Populacao (mi),Consumo médio per capita (kg),Razão Prod(mm)/Demanda,Razão Prod(mm)/População (kg/pct)
0,2018,Milho (em grão),2.048151e+06,Fubá de milho,Brasil,233514.28800,208494900,1.12,8.770988,9.823506
1,2019,Milho (em grão),2.154375e+06,Fubá de milho,Brasil,235364.78000,210147125,1.12,9.153345,10.251746
2,2020,Milho (em grão),2.247778e+06,Fubá de milho,Brasil,237166.37504,211755692,1.12,9.477641,10.614957
3,2021,Milho (em grão),2.287549e+06,Fubá de milho,Brasil,238915.75568,213317639,1.12,9.574708,10.723673
4,2022,Milho (em grão),2.194878e+06,Fubá de milho,Brasil,240607.96480,214828540,1.12,9.122218,10.216884
5,2023,Milho (em grão),2.226317e+06,Fubá de milho,Brasil,242238.38128,216284269,1.12,9.190604,10.293477
6,2024,Milho (em grão),2.261950e+06,Fubá de milho,Brasil,243806.59744,217684462,1.12,9.277642,10.390959
7,2025,Milho (em grão),2.283465e+06,Fubá de milho,Brasil,245312.58416,219029093,1.12,9.308391,10.425397
8,2026,Milho (em grão),2.290603e+06,Fubá de milho,Brasil,246754.51360,220316530,1.12,9.282922,10.396873
9,2027,Milho (em grão),2.291214e+06,Fubá de milho,Brasil,248130.66208,221545234,1.12,9.233900,10.341968


In [89]:
df_milho.columns

Index(['Ano', 'Item', 'Média móvel', 'Items', 'Região', 'Demanda pop (ton)',
       'Populacao (mi)', 'Consumo médio per capita (kg)',
       'Razão Prod(mm)/Demanda', 'Razão Prod(mm)/População (kg/pct)'],
      dtype='object')

In [134]:
fig =px.scatter(df_milho, x ='Ano', y = ['Média móvel', 'Demanda pop (ton)'], title = 'Projeção da produção/demanda por MILHO/FUBÁ DE MILHO - Brasil - (2018-2030)', labels= {'value': 'Produção (ton)'}, trendline = 'ols')
fig.update_layout(template = 'plotly_white')


fig.update_layout(
    font_family="Helvetica",
    font_color="black",
    title_font_family="Helvetica",
    title_font_color="black",
    legend_title_font_color="black")

* Soja

In [91]:
## Média móvel
df_soja['Média móvel'] = df_soja['Produçao'].rolling(window =5).mean()

In [92]:
fig =px.line(df_soja, x ='Ano', y=['Produçao', 'Média móvel'], title = 'Produção de SOJA - Brasil - (1974-2020)', labels= {'value': 'Produção (ton)'})
fig.update_layout(template = 'plotly_white')

In [93]:
df_soja.to_excel('df_soja_demanda.xlsx')

In [94]:
# Declarar as variáveis 
#Soja
Y_soja= df_soja.loc[4:]['Média móvel'].values  
X_soja = df_soja.loc[4:][['Ano']].values
X_soja= sm.add_constant(X_soja)

In [95]:
#Soja
modelo_soja = sm.OLS(Y_soja, X_soja)
resultados_soja = modelo_soja.fit()
#print(resultados_soja.summary())

In [96]:
print("Coeficientes: ", resultados_soja.params)
print("R2: ", resultados_soja.rsquared)

Coeficientes:  [-4.36941771e+09  2.20607155e+06]
R2:  0.8700789750330067


In [97]:
resultados_soja.params[1]

2206071.548051986

Filtrando os registros onde a coluna ANO é maior que 2018, já que estamos levando em consideração os padrões alimentares de consumo indicados pela POF (2018)

In [98]:
df_soja = pd.read_excel("C:\dcc-fellowship-ciencia-de-dados\previsões consumo e producao\df_soja_demanda.xlsx")
df_soja = df_soja[df_soja['Ano']>2017].reset_index(drop = True)
df_soja

,Ano,Item,Média móvel
0,2018,Soja (em grão),1.026530e+08
1,2019,Soja (em grão),1.081642e+08
2,2020,Soja (em grão),1.130308e+08
3,2021,Soja (em grão),1.152369e+08
4,2022,Soja (em grão),1.091061e+08
5,2023,Soja (em grão),1.118443e+08
6,2024,Soja (em grão),1.136825e+08
7,2025,Soja (em grão),1.147862e+08
8,2026,Soja (em grão),1.151373e+08
9,2027,Soja (em grão),1.151173e+08


Correção em relação a proporção da produção de SOJA destinada para a produção e consumo doméstico de ÓLEO DE SOJA

In [99]:
df_soja['Média móvel'] = df_soja['Média móvel']*0.075

In [100]:
df_soja

,Ano,Item,Média móvel
0,2018,Soja (em grão),7.698972e+06
1,2019,Soja (em grão),8.112317e+06
2,2020,Soja (em grão),8.477309e+06
3,2021,Soja (em grão),8.642764e+06
4,2022,Soja (em grão),8.182958e+06
5,2023,Soja (em grão),8.388319e+06
6,2024,Soja (em grão),8.526189e+06
7,2025,Soja (em grão),8.608963e+06
8,2026,Soja (em grão),8.635294e+06
9,2027,Soja (em grão),8.633800e+06


Demanda populacional por ÓLEO DE SOJA em toneladas (2018 -2030)

In [101]:
df_demanda_oleo_soja = projecao_demanda_brasil[projecao_demanda_brasil['Items'] == 'Óleo de soja'][['Ano', 'Items', 'Região', 'Demanda pop (ton)', 'Populacao (mi)', 'Consumo médio per capita (kg)']].reset_index(drop = True)
df_demanda_oleo_soja

,Ano,Items,Região,Demanda pop (ton),Populacao (mi),Consumo médio per capita (kg)
0,2018,Óleo de soja,Brasil,910427.730000,208494900,4.366667
1,2019,Óleo de soja,Brasil,917642.445833,210147125,4.366667
2,2020,Óleo de soja,Brasil,924666.521733,211755692,4.366667
3,2021,Óleo de soja,Brasil,931487.023633,213317639,4.366667
4,2022,Óleo de soja,Brasil,938084.624667,214828540,4.366667
5,2023,Óleo de soja,Brasil,944441.307967,216284269,4.366667
6,2024,Óleo de soja,Brasil,950555.484067,217684462,4.366667
7,2025,Óleo de soja,Brasil,956427.039433,219029093,4.366667
8,2026,Óleo de soja,Brasil,962048.847667,220316530,4.366667
9,2027,Óleo de soja,Brasil,967414.188467,221545234,4.366667


Juntando os dataframe df_soja e df_demanda_oleo_soja

In [102]:
df_soja = pd.merge(df_soja, df_demanda_oleo_soja, how = 'inner', on = 'Ano')
df_soja

,Ano,Item,Média móvel,Items,Região,Demanda pop (ton),Populacao (mi),Consumo médio per capita (kg)
0,2018,Soja (em grão),7.698972e+06,Óleo de soja,Brasil,910427.730000,208494900,4.366667
1,2019,Soja (em grão),8.112317e+06,Óleo de soja,Brasil,917642.445833,210147125,4.366667
2,2020,Soja (em grão),8.477309e+06,Óleo de soja,Brasil,924666.521733,211755692,4.366667
3,2021,Soja (em grão),8.642764e+06,Óleo de soja,Brasil,931487.023633,213317639,4.366667
4,2022,Soja (em grão),8.182958e+06,Óleo de soja,Brasil,938084.624667,214828540,4.366667
5,2023,Soja (em grão),8.388319e+06,Óleo de soja,Brasil,944441.307967,216284269,4.366667
6,2024,Soja (em grão),8.526189e+06,Óleo de soja,Brasil,950555.484067,217684462,4.366667
7,2025,Soja (em grão),8.608963e+06,Óleo de soja,Brasil,956427.039433,219029093,4.366667
8,2026,Soja (em grão),8.635294e+06,Óleo de soja,Brasil,962048.847667,220316530,4.366667
9,2027,Soja (em grão),8.633800e+06,Óleo de soja,Brasil,967414.188467,221545234,4.366667


In [103]:
df_soja['Razão Prod(mm)/Demanda'] = df_soja['Média móvel']/df_soja['Demanda pop (ton)']
df_soja['Razão Prod(mm)/População (kg/pct)'] = (df_soja['Média móvel']/df_soja['Populacao (mi)'])*1000
df_soja

,Ano,Item,Média móvel,Items,Região,Demanda pop (ton),Populacao (mi),Consumo médio per capita (kg),Razão Prod(mm)/Demanda,Razão Prod(mm)/População (kg/pct)
0,2018,Soja (em grão),7.698972e+06,Óleo de soja,Brasil,910427.730000,208494900,4.366667,8.456434,36.926430
1,2019,Soja (em grão),8.112317e+06,Óleo de soja,Brasil,917642.445833,210147125,4.366667,8.840390,38.603036
2,2020,Soja (em grão),8.477309e+06,Óleo de soja,Brasil,924666.521733,211755692,4.366667,9.167963,40.033439
3,2021,Soja (em grão),8.642764e+06,Óleo de soja,Brasil,931487.023633,213317639,4.366667,9.278459,40.515937
4,2022,Soja (em grão),8.182958e+06,Óleo de soja,Brasil,938084.624667,214828540,4.366667,8.723049,38.090648
5,2023,Soja (em grão),8.388319e+06,Óleo de soja,Brasil,944441.307967,216284269,4.366667,8.881779,38.783770
6,2024,Soja (em grão),8.526189e+06,Óleo de soja,Brasil,950555.484067,217684462,4.366667,8.969691,39.167650
7,2025,Soja (em grão),8.608963e+06,Óleo de soja,Brasil,956427.039433,219029093,4.366667,9.001171,39.305113
8,2026,Soja (em grão),8.635294e+06,Óleo de soja,Brasil,962048.847667,220316530,4.366667,8.975941,39.194944
9,2027,Soja (em grão),8.633800e+06,Óleo de soja,Brasil,967414.188467,221545234,4.366667,8.924616,38.970823


In [135]:
fig =px.scatter(df_soja, x ='Ano', y = ['Média móvel', 'Demanda pop (ton)'], title = 'Projeção da razão produção/demanda por SOJA/ÓLEO DE SOJA - Brasil - (2018-2030)', labels= {'value': 'Produção (ton)'}, trendline = 'ols')
fig.update_layout(template = 'plotly_white')


fig.update_layout(
    font_family="Helvetica",
    font_color="black",
    title_font_family="Helvetica",
    title_font_color="black",
    legend_title_font_color="black")

* Tomate

In [105]:
## Média móvel
df_tomate['Média móvel'] = df_tomate['Produçao'].rolling(window =5).mean()

In [106]:
fig =px.line(df_tomate, x ='Ano', y=['Produçao', 'Média móvel'], title = 'Produção de TOMATE - Brasil - (1974-2020)', labels= {'value': 'Produção (ton)'})
fig.update_layout(template = 'plotly_white')

In [107]:
df_tomate.to_excel('df_demanda_tomate.xlsx')

In [108]:
# Declarar as variáveis 
#Tomate
Y_tomate= df_tomate.loc[4:]['Média móvel'].values  
X_tomate = df_tomate.loc[4:][['Ano']].values
X_tomate= sm.add_constant(X_tomate)

In [109]:
#Tomate
modelo_tomate = sm.OLS(Y_tomate, X_tomate)
resultados_tomate = modelo_tomate.fit()
#print(resultados_tomate.summary())

In [110]:
print("Coeficientes: ", resultados_tomate.params)
print("R2: ", resultados_tomate.rsquared)

Coeficientes:  [-1.54504616e+08  7.87177928e+04]
R2:  0.9826947565947943


In [111]:
resultados_tomate.params[1]

78717.79278163797

Filtrando os registros onde a coluna ANO é maior que 2018, já que estamos levando em consideração os padrões alimentares de consumo indicados pela POF (2018)

In [112]:
df_tomate = pd.read_excel("C:\dcc-fellowship-ciencia-de-dados\previsões consumo e producao\df_tomate_demanda.xlsx")
df_tomate = df_tomate[df_tomate['Ano']>2017].reset_index(drop = True)
df_tomate

,Ano,Item,Média móvel
0,2018,Tomate,4.201939e+06
1,2019,Tomate,4.125583e+06
2,2020,Tomate,4.038757e+06
3,2021,Tomate,4.117474e+06
4,2022,Tomate,4.218282e+06
5,2023,Tomate,4.219124e+06
6,2024,Tomate,4.222561e+06
7,2025,Tomate,4.241956e+06
8,2026,Tomate,4.282596e+06
9,2027,Tomate,4.315621e+06


Correção em relação a proporção da produção de TOMATE destinada para a produção e consumo doméstico de MOLHO DE TOMATE

In [113]:
df_tomate['Média móvel'] = df_tomate['Média móvel']*0.37
df_tomate

,Ano,Item,Média móvel
0,2018,Tomate,1.554718e+06
1,2019,Tomate,1.526466e+06
2,2020,Tomate,1.494340e+06
3,2021,Tomate,1.523465e+06
4,2022,Tomate,1.560764e+06
5,2023,Tomate,1.561076e+06
6,2024,Tomate,1.562348e+06
7,2025,Tomate,1.569524e+06
8,2026,Tomate,1.584561e+06
9,2027,Tomate,1.596780e+06


Demanda populacional por MOLHO DE TOMATE em toneladas (2018 -2030)

In [114]:
df_demanda_molho_tomate = projecao_demanda_brasil[projecao_demanda_brasil['Items'] == 'Molho de tomate'][['Ano', 'Items', 'Região', 'Demanda pop (ton)', 'Populacao (mi)', 'Consumo médio per capita (kg)']].reset_index(drop = True)
df_demanda_molho_tomate

,Ano,Items,Região,Demanda pop (ton),Populacao (mi),Consumo médio per capita (kg)
0,2018,Molho de tomate,Brasil,233514.28800,208494900,1.12
1,2019,Molho de tomate,Brasil,235364.78000,210147125,1.12
2,2020,Molho de tomate,Brasil,237166.37504,211755692,1.12
3,2021,Molho de tomate,Brasil,238915.75568,213317639,1.12
4,2022,Molho de tomate,Brasil,240607.96480,214828540,1.12
5,2023,Molho de tomate,Brasil,242238.38128,216284269,1.12
6,2024,Molho de tomate,Brasil,243806.59744,217684462,1.12
7,2025,Molho de tomate,Brasil,245312.58416,219029093,1.12
8,2026,Molho de tomate,Brasil,246754.51360,220316530,1.12
9,2027,Molho de tomate,Brasil,248130.66208,221545234,1.12


Juntando os dataframe df_tomate e df_molho_tomate

In [115]:
df_tomate = pd.merge(df_tomate, df_demanda_molho_tomate, how = 'inner', on = 'Ano')
df_tomate

,Ano,Item,Média móvel,Items,Região,Demanda pop (ton),Populacao (mi),Consumo médio per capita (kg)
0,2018,Tomate,1.554718e+06,Molho de tomate,Brasil,233514.28800,208494900,1.12
1,2019,Tomate,1.526466e+06,Molho de tomate,Brasil,235364.78000,210147125,1.12
2,2020,Tomate,1.494340e+06,Molho de tomate,Brasil,237166.37504,211755692,1.12
3,2021,Tomate,1.523465e+06,Molho de tomate,Brasil,238915.75568,213317639,1.12
4,2022,Tomate,1.560764e+06,Molho de tomate,Brasil,240607.96480,214828540,1.12
5,2023,Tomate,1.561076e+06,Molho de tomate,Brasil,242238.38128,216284269,1.12
6,2024,Tomate,1.562348e+06,Molho de tomate,Brasil,243806.59744,217684462,1.12
7,2025,Tomate,1.569524e+06,Molho de tomate,Brasil,245312.58416,219029093,1.12
8,2026,Tomate,1.584561e+06,Molho de tomate,Brasil,246754.51360,220316530,1.12
9,2027,Tomate,1.596780e+06,Molho de tomate,Brasil,248130.66208,221545234,1.12


In [116]:
df_tomate['Razão Prod(mm)/Demanda'] = df_tomate['Média móvel']/df_tomate['Demanda pop (ton)']
df_tomate['Razão Prod(mm)/População (kg/pct)'] = (df_tomate['Média móvel']/df_tomate['Populacao (mi)'])*1000
df_tomate

,Ano,Item,Média móvel,Items,Região,Demanda pop (ton),Populacao (mi),Consumo médio per capita (kg),Razão Prod(mm)/Demanda,Razão Prod(mm)/População (kg/pct)
0,2018,Tomate,1.554718e+06,Molho de tomate,Brasil,233514.28800,208494900,1.12,6.657912,7.456861
1,2019,Tomate,1.526466e+06,Molho de tomate,Brasil,235364.78000,210147125,1.12,6.485532,7.263796
2,2020,Tomate,1.494340e+06,Molho de tomate,Brasil,237166.37504,211755692,1.12,6.300809,7.056906
3,2021,Tomate,1.523465e+06,Molho de tomate,Brasil,238915.75568,213317639,1.12,6.376579,7.141769
4,2022,Tomate,1.560764e+06,Molho de tomate,Brasil,240607.96480,214828540,1.12,6.486752,7.265163
5,2023,Tomate,1.561076e+06,Molho de tomate,Brasil,242238.38128,216284269,1.12,6.444379,7.217704
6,2024,Tomate,1.562348e+06,Molho de tomate,Brasil,243806.59744,217684462,1.12,6.408143,7.177120
7,2025,Tomate,1.569524e+06,Molho de tomate,Brasil,245312.58416,219029093,1.12,6.398057,7.165824
8,2026,Tomate,1.584561e+06,Molho de tomate,Brasil,246754.51360,220316530,1.12,6.421608,7.192200
9,2027,Tomate,1.596780e+06,Molho de tomate,Brasil,248130.66208,221545234,1.12,6.435237,7.207466


In [136]:
fig =px.scatter(df_tomate, x ='Ano', y = ['Média móvel', 'Demanda pop (ton)'], title = 'Projeção da razão produção/demanda por TOMATE/MOLHO DE TOMATE - Brasil - (2018-2030)', labels= {'value': 'Produção (ton)'}, trendline = 'ols')
fig.update_layout(template = 'plotly_white')


fig.update_layout(
    font_family="Helvetica",
    font_color="black",
    title_font_family="Helvetica",
    title_font_color="black",
    legend_title_font_color="black")

* Trigo

In [118]:
## Média móvel
df_trigo['Média móvel'] = df_trigo['Produçao'].rolling(window =5).mean()

In [119]:
fig =px.line(df_trigo, x ='Ano', y=['Produçao', 'Média móvel'], title = 'Produção de TRIGO - Brasil - (1974-2020)', labels= {'value': 'Produção (ton)'})
fig.update_layout(template = 'plotly_white')

In [120]:
df_trigo.to_excel('df_trigo_demanda.xlsx')

In [121]:
# Declarar as variáveis 
#Tomate
Y_trigo= df_tomate.loc[4:]['Média móvel'].values  
X_trigo = df_tomate.loc[4:][['Ano']].values
X_trigo= sm.add_constant(X_trigo)

In [122]:
#Trigo
modelo_trigo = sm.OLS(Y_trigo, X_trigo)
resultados_trigo = modelo_trigo.fit()
#print(resultados_trigo.summary())

In [123]:
print("Coeficientes: ", resultados_trigo.params)
print("R2: ", resultados_trigo.rsquared)

Coeficientes:  [-1.57148189e+07  8.53940859e+03]
R2:  0.9560597917744066


In [124]:
resultados_trigo.params[1]

8539.408594247463

Filtrando os registros onde a coluna ANO é maior que 2018, já que estamos levando em consideração os padrões alimentares de consumo indicados pela POF (2018)

In [125]:
df_trigo = pd.read_excel("C:\dcc-fellowship-ciencia-de-dados\previsões consumo e producao\df_trigo_demanda.xlsx")
df_trigo = df_trigo[df_trigo['Ano']>2017].reset_index(drop = True)
df_trigo

,Ano,Item,Média móvel
0,2018,Trigo (em grão),5.683363e+06
1,2019,Trigo (em grão),5.549147e+06
2,2020,Trigo (em grão),5.717054e+06
3,2021,Trigo (em grão),5.740133e+06
4,2022,Trigo (em grão),5.708461e+06
5,2023,Trigo (em grão),5.702711e+06
6,2024,Trigo (em grão),5.706580e+06
7,2025,Trigo (em grão),5.738067e+06
8,2026,Trigo (em grão),5.742269e+06
9,2027,Trigo (em grão),5.742696e+06


Demanda populacional por FARINHA DE TRIGO em toneladas (2018 -2030)

In [126]:
df_demanda_farinha_trigo = projecao_demanda_brasil[projecao_demanda_brasil['Items'] == 'Farinha de trigo'][['Ano', 'Items', 'Região', 'Demanda pop (ton)', 'Populacao (mi)', 'Consumo médio per capita (kg)']].reset_index(drop = True)
df_demanda_farinha_trigo

,Ano,Items,Região,Demanda pop (ton),Populacao (mi),Consumo médio per capita (kg)
0,2018,Farinha de trigo,Brasil,481970.710500,208494900,2.311667
1,2019,Farinha de trigo,Brasil,485790.103958,210147125,2.311667
2,2020,Farinha de trigo,Brasil,489508.574673,211755692,2.311667
3,2021,Farinha de trigo,Brasil,493119.275488,213317639,2.311667
4,2022,Farinha de trigo,Brasil,496611.974967,214828540,2.311667
5,2023,Farinha de trigo,Brasil,499977.135172,216284269,2.311667
6,2024,Farinha de trigo,Brasil,503213.914657,217684462,2.311667
7,2025,Farinha de trigo,Brasil,506322.253318,219029093,2.311667
8,2026,Farinha de trigo,Brasil,509298.378517,220316530,2.311667
9,2027,Farinha de trigo,Brasil,512138.732597,221545234,2.311667


Juntando os dataframe df_trigo e df_farinha_trigo

In [127]:
df_trigo = pd.merge(df_trigo, df_demanda_farinha_trigo, how = 'inner', on = 'Ano')
df_trigo

,Ano,Item,Média móvel,Items,Região,Demanda pop (ton),Populacao (mi),Consumo médio per capita (kg)
0,2018,Trigo (em grão),5.683363e+06,Farinha de trigo,Brasil,481970.710500,208494900,2.311667
1,2019,Trigo (em grão),5.549147e+06,Farinha de trigo,Brasil,485790.103958,210147125,2.311667
2,2020,Trigo (em grão),5.717054e+06,Farinha de trigo,Brasil,489508.574673,211755692,2.311667
3,2021,Trigo (em grão),5.740133e+06,Farinha de trigo,Brasil,493119.275488,213317639,2.311667
4,2022,Trigo (em grão),5.708461e+06,Farinha de trigo,Brasil,496611.974967,214828540,2.311667
5,2023,Trigo (em grão),5.702711e+06,Farinha de trigo,Brasil,499977.135172,216284269,2.311667
6,2024,Trigo (em grão),5.706580e+06,Farinha de trigo,Brasil,503213.914657,217684462,2.311667
7,2025,Trigo (em grão),5.738067e+06,Farinha de trigo,Brasil,506322.253318,219029093,2.311667
8,2026,Trigo (em grão),5.742269e+06,Farinha de trigo,Brasil,509298.378517,220316530,2.311667
9,2027,Trigo (em grão),5.742696e+06,Farinha de trigo,Brasil,512138.732597,221545234,2.311667


In [128]:
df_trigo['Razão Prod(mm)/Demanda'] = df_trigo['Média móvel']/df_trigo['Demanda pop (ton)']
df_trigo['Razão Prod(mm)/População (kg/pct)'] = (df_trigo['Média móvel']/df_trigo['Populacao (mi)'])*1000
df_trigo

,Ano,Item,Média móvel,Items,Região,Demanda pop (ton),Populacao (mi),Consumo médio per capita (kg),Razão Prod(mm)/Demanda,Razão Prod(mm)/População (kg/pct)
0,2018,Trigo (em grão),5.683363e+06,Farinha de trigo,Brasil,481970.710500,208494900,2.311667,11.791926,27.259002
1,2019,Trigo (em grão),5.549147e+06,Farinha de trigo,Brasil,485790.103958,210147125,2.311667,11.422931,26.406010
2,2020,Trigo (em grão),5.717054e+06,Farinha de trigo,Brasil,489508.574673,211755692,2.311667,11.679171,26.998350
3,2021,Trigo (em grão),5.740133e+06,Farinha de trigo,Brasil,493119.275488,213317639,2.311667,11.640456,26.908854
4,2022,Trigo (em grão),5.708461e+06,Farinha de trigo,Brasil,496611.974967,214828540,2.311667,11.494811,26.572170
5,2023,Trigo (em grão),5.702711e+06,Farinha de trigo,Brasil,499977.135172,216284269,2.311667,11.405943,26.366738
6,2024,Trigo (em grão),5.706580e+06,Farinha de trigo,Brasil,503213.914657,217684462,2.311667,11.340267,26.214917
7,2025,Trigo (em grão),5.738067e+06,Farinha de trigo,Brasil,506322.253318,219029093,2.311667,11.332835,26.197738
8,2026,Trigo (em grão),5.742269e+06,Farinha de trigo,Brasil,509298.378517,220316530,2.311667,11.274863,26.063724
9,2027,Trigo (em grão),5.742696e+06,Farinha de trigo,Brasil,512138.732597,221545234,2.311667,11.213166,25.921101


In [129]:
fig =px.scatter(df_trigo, x ='Ano', y = ['Razão Prod(mm)/Demanda', 'Razão Prod(mm)/População (kg/pct)'], title = 'Projeção da razão produção/demanda por TOMATE/MOLHO DE TOMATE - Brasil - (2018-2030)', labels= {'value': 'Produção (ton)'}, trendline = 'ols')
fig.update_layout(template = 'plotly_white')